In [1]:
# Importing necessary libraries
import pandas as pd
from utils import *


In [2]:
# Hyperparameters and paths

first_execution = False  # Set to True for the first execution to split the excel file

RAW = Path("data/raw/Observations_2012-2025.xlsx")
OUT_FILT = Path("data/filtered")
OUT_CLEAN = Path("data/filtered/cleaned")

In [3]:
# Run on the first execution to split and save the excel file into multiple csv files
if first_execution:
    print(split_excel_to_csvs(RAW, OUT_FILT))

In [4]:
# For following executions, load the csv files directly
dfs = load_csvs(OUT_FILT)
for k, df in dfs.items():
    print(f"{k}: {df.shape}")


ESPECES: (86, 5)
GPS-MILIEU: (651, 8)
NOM FRANÇAIS: (114497, 26)


In [ ]:
# Visual inspection of dataframes, uncoment as needed
# dfs['ESPECES']
# dfs['GPS-MILIEU'].head()
# dfs['NOM FRANÇAIS'].head()

,Nom observateur,code département,Nom transect,date,"1er, 2e ou 3e passage",nuages,pluie,vent,visibilité,N° point,...,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,totaux,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<100m,NaN,> 100m,NaN,vol,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Auditif,Visuel,Auditif,Visuel,NaN,Auditif,"Visuel (sans""Vol"")","A+V (sans ""Vol"")","A+V (avec ""Vol"")",NaN
2,BELFAN David,972.0,Fond l'Etang,2014-04-12 00:00:00,1.0,2,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,1,0,1,1,NaN
3,BELFAN David,972.0,Fond l'Etang,2014-04-12 00:00:00,1.0,2,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,0,1,1,1,NaN
4,BELFAN David,972.0,Fond l'Etang,2014-04-12 00:00:00,1.0,2,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,0,2,2,2,NaN


In [ ]:
# Cleaning dataframes from initial loading issues
clean_especes = clean_especes(dfs["ESPECES"])
clean_gps = clean_gps(dfs["GPS-MILIEU"])
clean_observations = clean_observations(dfs["NOM FRANÇAIS"])

# Saving cleaned dataframes back to csv files
save_clean_csvs({"ESPECES": clean_especes, "GPS-MILIEU": clean_gps, "NOM FRANÇAIS": clean_observations}, OUT_CLEAN)

# uncomment as needed to inspect cleaned dataframes
# clean_especes.head()
# clean_gps.head()
# clean_observations.head()

{'especes_clean': 'data/filtered/cleaned/especes_clean.csv',
 'gps_milieu_clean': 'data/filtered/cleaned/gps_milieu_clean.csv',
 'nom_francais_clean': 'data/filtered/cleaned/nom_francais_clean.csv'}